<a href="https://colab.research.google.com/github/TheGreatWaves/C-base/blob/main/Copy_of_Project_2_Non_DL_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import numpy as np
from numpy import ndarray
import pandas as pd

import librosa

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa.display
from tqdm import tqdm


import glob
import torch

from pathlib import Path

In [2]:
librosa.__version__

'0.10.1'

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
# # Assumptions:
# # - The transcription file is located in the same directory as the audio files.
# class AudioDataset:
#   """
#   Class for loading and storing audio data.
#   """

#   def __init__(self):
#     self.df = pd.DataFrame(columns=['id', 'path', 'transcription'])

#   def load_transcriptions(self, directory_path: str) -> bool:
#     """
#     Load all transcriptions from a given directory, including subdirectories.
#     Returns False if no transcription files were found, or if any failed to load.
#     """
#     transcriptions_path = glob.glob(
#         f"{directory_path}/**/*.trans.txt",
#         recursive=True
#     )

#     if len(transcriptions_path) == 0:
#       return False

#     for path in transcriptions_path:
#       if not self.load_transcription_file(path):
#         return False

#     return True



#   def load_transcription_file(self, file_path: str) -> bool:
#     """
#     Parse transcription file and records the audio ID - subtitle mapping.
#     Returns False if the file could not be read.
#     """
#     with open(file_path, "r") as file:
#       file_directory = os.path.dirname(file_path)

#       lines = file.read().split("\n")
#       for line in lines:
#         if len(line.strip()) == 0:
#           continue
#         splitter = line.split(" ")
#         file_name = splitter[0]
#         file_content = ' '.join(splitter[1:])
#         self.df.loc[len(self.df)] = {
#             'id':file_name,
#             'transcription':file_content,
#             'path': f'{file_directory}/{file_name}.flac'
#         }
#       return True
#     return False

#   def keys(self):
#     return iter(self.df['id'])

#   def get(self, id: int):
#     """
#     Retrieve a dataframe row from ID.
#     """
#     return self.df.loc[self.df['id'] == id]

In [6]:
# Assumptions:
# - The transcription file is located in the same directory as the audio files.
class AudioDataset:
  """
  Class for loading and storing audio data.
  """

  def __init__(self):
    self.df = pd.DataFrame(columns=['id', 'path', 'transcription'])

  def load_transcriptions(self, directory_path: str) -> bool:
    """
    Load all transcriptions from a given directory, including subdirectories.
    Returns False if no transcription files were found, or if any failed to load.
    """
    sound_names = glob.glob(
        f"{directory_path}/**/*.wav",
        recursive=True
    )

    if len(sound_names) == 0:
      return False

    for path in sound_names:
      if not self.load_transcription_file(path):
        return False

    return True



  def load_transcription_file(self, file_path: str) -> bool:
    """
    Parse transcription file and records the audio ID - subtitle mapping.
    Returns False if the file could not be read.
    """

    file_directory = Path(file_path).parent
    file_name = Path(file_path).stem
    self.df.loc[len(self.df)] = {
        'id':file_name,
        'transcription':file_directory.name,
        'path': f'{file_directory}/{file_name}.wav'
    }
    return True

  def keys(self):
    return iter(self.df['id'])

  def get(self, id: int):
    """
    Retrieve a dataframe row from ID.
    """
    return self.df.loc[self.df['id'] == id]

In [7]:
df_dict = {}

In [8]:
# ds = AudioDataset()

In [9]:
# all_subdirectories = glob.glob(
#         f"/content/drive/MyDrive/test_hmm/*/",
#         recursive=True
# )

# for path in all_subdirectories:
#   fruit_label = Path(path).name
#   ds = AudioDataset()
#   if ds.load_transcriptions(path):
#     df_dict[fruit_label] = ds.df


In [10]:
all_subdirectories = glob.glob(
        f"/content/drive/MyDrive/datasets/single_words/*",
        recursive=True
)

for path in all_subdirectories:
  print(path)
  fruit_label = Path(path).name
  ds = AudioDataset()
  if ds.load_transcriptions(path):
    df_dict[fruit_label] = ds.df

/content/drive/MyDrive/datasets/single_words/right
/content/drive/MyDrive/datasets/single_words/one
/content/drive/MyDrive/datasets/single_words/seven
/content/drive/MyDrive/datasets/single_words/sheila
/content/drive/MyDrive/datasets/single_words/six
/content/drive/MyDrive/datasets/single_words/stop
/content/drive/MyDrive/datasets/single_words/three
/content/drive/MyDrive/datasets/single_words/tree
/content/drive/MyDrive/datasets/single_words/two
/content/drive/MyDrive/datasets/single_words/up
/content/drive/MyDrive/datasets/single_words/yes
/content/drive/MyDrive/datasets/single_words/wow
/content/drive/MyDrive/datasets/single_words/zero
/content/drive/MyDrive/datasets/single_words/bed
/content/drive/MyDrive/datasets/single_words/bird
/content/drive/MyDrive/datasets/single_words/cat
/content/drive/MyDrive/datasets/single_words/dog
/content/drive/MyDrive/datasets/single_words/down
/content/drive/MyDrive/datasets/single_words/eight
/content/drive/MyDrive/datasets/single_words/five
/con

In [11]:
df_dict

{'apple':          id                                               path transcription
 0   apple04  /content/drive/MyDrive/test_hmm/apple/apple04.wav         apple
 1   apple08  /content/drive/MyDrive/test_hmm/apple/apple08.wav         apple
 2   apple15  /content/drive/MyDrive/test_hmm/apple/apple15.wav         apple
 3   apple14  /content/drive/MyDrive/test_hmm/apple/apple14.wav         apple
 4   apple12  /content/drive/MyDrive/test_hmm/apple/apple12.wav         apple
 5   apple10  /content/drive/MyDrive/test_hmm/apple/apple10.wav         apple
 6   apple05  /content/drive/MyDrive/test_hmm/apple/apple05.wav         apple
 7   apple11  /content/drive/MyDrive/test_hmm/apple/apple11.wav         apple
 8   apple09  /content/drive/MyDrive/test_hmm/apple/apple09.wav         apple
 9   apple02  /content/drive/MyDrive/test_hmm/apple/apple02.wav         apple
 10  apple07  /content/drive/MyDrive/test_hmm/apple/apple07.wav         apple
 11  apple06  /content/drive/MyDrive/test_hmm/apple/app

### Extracting Features

In [12]:
# from librosa.feature import mfcc
# import librosa
# audio, sampling_freq = librosa.load(ds.df.head()['path'].values[0])
# mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
# print('\nNumber of windows =', mfcc_features.shape[0])
# print('Length of each feature =', mfcc_features.shape[1])

In [13]:
# mfcc_features = mfcc_features.T
# plt.matshow(mfcc_features)
# plt.title('MFCC')

In [14]:
!pip install hmmlearn
!pip install features

In [15]:
import os
import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm #importing GaussianHMM
import librosa # reading wavefilesfrom librosa.feature import mfcc #to extract mfcc features

In [16]:
class HMMTrainer(object):
  def __init__(self, model_name='GaussianHMM', n_components=4):
     self.model_name = model_name
     self.n_components = n_components

     self.models = []
     if self.model_name == 'GaussianHMM':
        self.model=hmm.GaussianHMM(n_components=4)
     else:
        print("Please choose GaussianHMM")

  def train(self, X):
      self.models.append(self.model.fit(X))

  def get_score(self, input_data):
      return self.model.score(input_data)

In [17]:
# ds.df

In [18]:
hmm_models = []

for df in df_dict.items():
  fruit_label = df[0]
  X = np.array([])
  for index, row in df[1].iloc[:-1,:].iterrows():
    # Read the input file
    audio, sampling_freq = librosa.load(row['path'])
    # Extract MFCC features
    mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
    # Append to the variable X
    if len(X) == 0:
      X = mfcc_features[:, :15]  # 15 here denotes the number of MFCC coefficients to consider
    else:
      X = np.append(X, mfcc_features[:, :15], axis=0)
    # Append the label
    # print('X.shape =', X.shape)

  # Train HMM model for this iteration
  hmm_trainer = HMMTrainer()
  hmm_trainer.train(X)  # Train using the current MFCC features
  hmm_models.append((hmm_trainer, fruit_label))


In [31]:
print(len(hmm_models))

37


In [32]:
test_files = {}

for df in df_dict.items():
  truth_label = df[0]
  selected_test = df[1].iloc[-1,:]
  test_files[truth_label] = selected_test['path']

In [21]:
print(test_files)

{'apple': '/content/drive/MyDrive/test_hmm/apple/apple01.wav', 'banana': '/content/drive/MyDrive/test_hmm/banana/banana13.wav', 'lime': '/content/drive/MyDrive/test_hmm/lime/lime09.wav', 'orange': '/content/drive/MyDrive/test_hmm/orange/orange01.wav', 'kiwi': '/content/drive/MyDrive/test_hmm/kiwi/kiwi02.wav', 'pineapple': '/content/drive/MyDrive/test_hmm/pineapple/pineapple07.wav', 'peach': '/content/drive/MyDrive/test_hmm/peach/peach03.wav', 'right': '/content/drive/MyDrive/datasets/single_words/right/18.wav', 'one': '/content/drive/MyDrive/datasets/single_words/one/4.wav', 'seven': '/content/drive/MyDrive/datasets/single_words/seven/27.wav', 'sheila': '/content/drive/MyDrive/datasets/single_words/sheila/22.wav', 'six': '/content/drive/MyDrive/datasets/single_words/six/3.wav', 'stop': '/content/drive/MyDrive/datasets/single_words/stop/3.wav', 'three': '/content/drive/MyDrive/datasets/single_words/three/17.wav', 'tree': '/content/drive/MyDrive/datasets/single_words/tree/3.wav', 'two': 

In [33]:
for truth_label, path in test_files.items():
  audio, sampling_freq = librosa.load(path)
  # Extract MFCC features
  mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
  X = mfcc_features[:, :15]
  scores = []
  for hmm_model, label in hmm_models:
    score = hmm_model.get_score(X)
    scores.append(score)
  index = np.array(scores).argmax()
  # Print the output
  print("\nTrue:", truth_label)
  print("Predicted:", hmm_models[index][1])


True: apple
Predicted: pineapple

True: banana
Predicted: banana

True: lime
Predicted: lime

True: orange
Predicted: lime

True: kiwi
Predicted: kiwi

True: pineapple
Predicted: pineapple

True: peach
Predicted: left

True: right
Predicted: right

True: one
Predicted: three

True: seven
Predicted: seven

True: sheila
Predicted: sheila

True: six
Predicted: six

True: stop
Predicted: stop

True: three
Predicted: three

True: tree
Predicted: stop

True: two
Predicted: two

True: up
Predicted: up

True: yes
Predicted: yes

True: wow
Predicted: nine

True: zero
Predicted: yes

True: bed
Predicted: three

True: bird
Predicted: off

True: cat
Predicted: three

True: dog
Predicted: up

True: down
Predicted: wow

True: eight
Predicted: up

True: five
Predicted: five

True: four
Predicted: three

True: go
Predicted: house

True: happy
Predicted: happy

True: house
Predicted: house

True: left
Predicted: left

True: marvel
Predicted: marvel

True: nine
Predicted: left

True: no
Predicted: no



In [23]:
# hmm_models = []
# X = np.array([])

# for index, row in ds.df.iterrows():
#    # Read the input file
#    audio, sampling_freq = librosa.load(row['path'])
#    # Extract MFCC features
#    mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
#    # Append to the variable X
#    if len(X) == 0:
#      X = mfcc_features[:, :15]  # 15 here denotes the number of MFCC coefficients to consider
#    else:
#      X = np.append(X, mfcc_features[:, :15], axis=0)
#    # Append the label
#   #  print('X.shape =', X.shape)

#    # Train HMM model for this iteration
#    hmm_trainer = HMMTrainer()
#    hmm_trainer.train(X)  # Train using the current MFCC features
#    hmm_models.append((hmm_trainer, row['transcription']))
#    hmm_trainer = None


In [24]:
# test = ds.df.iloc[-1,:]
# # Read the input file
# audio, sampling_freq = librosa.load(test['path'])
# # Extract MFCC features
# mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
# X = mfcc_features[:, :15]
# y_word = test['transcription']

In [25]:
# scores = []
# for hmm_model, _ in hmm_models:
#     score = hmm_model.get_score(X)
#     scores.append(score)

# index = np.array(scores).argmax()
# # Print the output
# print("\nTrue:", y_word)
# print("Predicted:", hmm_models[index][1])

In [26]:
!pip install torchaudio ipywebrtc

In [27]:
# Download a static FFmpeg build and add it to PATH.
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
print('')
!which ffmpeg
print('Done!')


/usr/bin/ffmpeg
Done!


In [28]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

from google.colab import output
output.enable_custom_widget_manager()

In [34]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [36]:
# Write to recording.webm
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Translate recording.wemb to file.wav
!ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic

# Extract features from input and predict
input_audio, sr = librosa.load('file.wav')
mfcc_features = librosa.feature.mfcc(sr=sr, y=input_audio)
X = mfcc_features[:, :15]
scores = []
for hmm_model, label in hmm_models:
  score = hmm_model.get_score(X)
  scores.append(score)
index = np.array(scores).argmax()
print("Predicted:", hmm_models[index][1])

Predicted: six
